In [8]:
# Import necessary libraries
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from matplotlib import pyplot as plt
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the image
image_path = '/content/drive/MyDrive/DeepLearning/Earth.jpeg'

# Load the image using PIL
image = Image.open(image_path)
image_np = np.array(image)

# Download and import the TensorFlow Object Detection API
!git clone --depth 1 https://github.com/tensorflow/models.git
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# Load the pre-trained model and label map from a different source
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
model_dir = tf.keras.utils.get_file(
  fname=model_name,
  origin='http://download.tensorflow.org/models/object_detection/' + model_name + '.tar.gz',
  untar=True
)
model = tf.saved_model.load(os.path.join(model_dir, 'saved_model'))
model = model.signatures['serving_default']

# Download the label map
!wget https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt

label_map_path = 'mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(label_map_path, use_display_name=True)

# Run inference
input_tensor = tf.convert_to_tensor(image_np)
input_tensor = input_tensor[tf.newaxis,...]
output_dict = model(input_tensor)

# Print output_dict
print(output_dict)

# Visualization
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    np.squeeze(output_dict['detection_boxes']),
    np.squeeze(output_dict['detection_classes']).astype(np.int32),
    np.squeeze(output_dict['detection_scores']),
    category_index,
    use_normalized_coordinates=True,
    max_boxes_to_draw=200,
    min_score_thresh=0.30,
    agnostic_mode=False)

# Display the image with detected objects
plt.figure(figsize=(12, 8))
plt.imshow(image_np)
plt.axis('off')
plt.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into 'models'...
remote: Enumerating objects: 4084, done.
remote: Counting objects: 100% (4084/4084), done.
remote: Compressing objects: 100% (3074/3074), done.
remote: Total 4084 (delta 1189), reused 2904 (delta 950), pack-reused 0
Receiving objects: 100% (4084/4084), 44.61 MiB | 22.57 MiB/s, done.
Resolving deltas: 100% (1189/1189), done.
/content/models/research/models/research/models/research/models/research/models/research
--2024-03-06 10:26:20--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5056 (4.9K) [